In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.decomposition import PCA




In [98]:
dataset = pd.read_csv("Hotel Reservations.csv")

In [99]:
dataset

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.50,0,Canceled


In [100]:
dataset.rename(columns={'arrival_year':'year','arrival_month':'month','arrival_date':'day'},inplace=True)
dataset['arrival'] = pd.to_datetime(dataset[['day','month','year']],errors='coerce')

The dataset has 37 entries with invalid data (29/02/2018)

In [101]:
dataset.drop(index=dataset[dataset.arrival.isna()].index,inplace=True)
dataset.arrival = dataset.arrival.values.astype(np.int64) // 10 ** 9

one hot encode meal plan and room type and market segment and binary code booking status

In [102]:
dataset = pd.get_dummies(dataset,columns=['type_of_meal_plan','room_type_reserved','market_segment_type'])

In [103]:
booking_stat = {'Not_Canceled':0,'Canceled':1}
dataset.booking_status = [booking_stat[item] for item in dataset.booking_status]

drop useless columns

In [104]:
dataset.drop(columns=['year','day','month','Booking_ID'],inplace=True)

Classifiers

In [105]:
classifiers = [
    KNeighborsClassifier(5),
    SVC(),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=32),
    RandomForestClassifier(max_depth=32, n_estimators=100),
    MLPClassifier(learning_rate='adaptive',max_iter=1000),
    AdaBoostClassifier(n_estimators=50),
    GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

names = [
    "Nearest Neighbors",
    "SVC",
    "Linear SVM",
    "RBF SVM",
    #"Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net (MLP)",
    "AdaBoost",
    "GBoost",
    "Naive Bayes",
    "QDA",
]

In [106]:
X_data = dataset.drop(["booking_status"],axis=1)
y_data = dataset.booking_status

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3,shuffle=True)

In [107]:
for name, clf in zip(names,classifiers):
    clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("{name}:{score:.2f}".format(name=name,score=score))


Nearest Neighbors:0.85
SVC:0.83
Linear SVM:0.80
RBF SVM:0.86
Decision Tree:0.87
Random Forest:0.90
Neural Net (MLP):0.85
AdaBoost:0.82
GBoost:0.83
Naive Bayes:0.41
QDA:0.36


c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
